### Vergleich mit Wegahome
#### Gemessene Daten von 3 Kunden, Standorten: Essen, Bochum und Leverkusen

In [1]:
import pandas as pd

csv_essen = pd.read_csv('./Essen_2024.csv')
# Konvertiere die 'measuredAt'-Spalte in ein datetime-Format
csv_essen['measuredAt'] = pd.to_datetime(csv_essen['measuredAt'])
csv_essen.set_index('measuredAt', inplace=True)

df_essen = csv_essen.resample('1h').mean()
teilen = ['PV (Wh)', 'Gesamte Netzeinspeisung (Wh)', 'Gesamter Netzbezug (Wh)', 'Batterieladung (Wh)', 'EV (Wh)', 'Haushalt (Wh)', 'Verbrauch (Wh)', 'Wärmepumpe (Wh)']  


In [2]:
# Summe in kW
pv_ertrag= round(df_essen['PV (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
einspeisung = round(df_essen['Gesamte Netzeinspeisung (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
netzbezug = round(df_essen['Gesamter Netzbezug (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
strombedarf = round(df_essen['Haushalt (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
wp_strom = round(df_essen['Wärmepumpe (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
bs_strom = round(df_essen['Batterieladung (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
ev_strom = round(df_essen['EV (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
verbrauch = round(df_essen['Verbrauch (Wh)'].fillna(0).astype(float).sum() / 1000, 2)

### Simulation mit HEMS-Rechner

In [3]:
# Haus ähnlich zu Essen simulieren
plz = 45257
anlage_groesse = 7.6
battery_capacity = 10.2
heizlast = 7 # eigetlich WP-Größe
homeoffice = True
flaeche = 100
anzahl_personen = 1
strompreis = 0.358

In [4]:
import berechnen_wp, pv_profil, lastprofile_VDI4655, temperatur_aussen, try_region, heizkurve

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', flaeche, anzahl_personen, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(plz)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, flaeche, TRY_region, 2)
df['T_aussen'] = temperatur_aussen.get_hourly_temperature(plz, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df['T_aussen'], T_n_aussen)
df['T_vor'] = hz['T_vor']
wp_groesse, wp_modell = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df = heizkurve.get_cop(wp_groesse, df)
V_ps, PS_verlust, Q_ps, Q_ps_max, Q_ps_ueber = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
pv = pv_profil.get_pv_profil(plz, 2014, anlage_groesse)

In [5]:
df_wp = berechnen_wp.get_max_heizleistung(wp_groesse, df.copy())
df_1, df_ohne = berechnen_wp.mit_hems(df_wp.copy(), pv, Q_ps, Q_ps_max, Q_ps_ueber, PS_verlust)
df_2, df_ohne_2 = berechnen_wp.mit_hems_bsev(df_1, df_ohne, pv, battery_capacity, anlage_groesse, homeoffice)


In [6]:
start_date = "2014-03-26 00:00:00"
end_date = "2014-11-30 23:00:00"

df_filtered = df_2.loc[start_date:end_date]
df_ohne_filtered = df_ohne_2.loc[start_date:end_date]

ersparnis = berechnen_wp.ersparnis_hems_bsev(df_filtered, df_ohne_filtered, anlage_groesse, strompreis)
berechnen_wp.print_ersparnis_hems(ersparnis)

Haushaltsstrombedarf in kWh: 1572
Jahresertrag in kWh: 6762

Ohne HEMS
Wärmepumpe Strombedarf in kWh: 1159
EV Strombedarf in kWh: 1343

Geladene PV-Strom in Wärmepumpe in kWh: 481
Geladene PV-Strom in Batteriespeicher in kWh: 1584
Geladene PV-Strom in Elektroauto in kWh: 111

Geladene BS-Strom in Wärmepumpe in kWh: 297
Geladene BS-Strom in Elektroauto in kWh: 701

Netzbezug in kWh: 1191
Stromkosten in €/a: 426.38
Einspeisung ins Netz in kWh: 3750
Einspeisevergütung in €/a: 298.5
Eigenverbrauch in kWh: 3012
kg CO₂/a: 452.58

Mit HEMS
Wärmepumpe Strombedarf in kWh: 1159
EV Strombedarf in kWh: 1307

Geladene PV-Strom in Wärmepumpe in kWh: 501
Geladene PV-Strom in Batteriespeicher in kWh: 1098
Geladene PV-Strom in Elektroauto in kWh: 1129

Geladene BS-Strom in Wärmepumpe in kWh: 359
Geladene BS-Strom in Elektroauto in kWh: 59

Netzbezug in kWh: 580
Stromkosten in €/a: 207.64
Einspeisung ins Netz in kWh: 3219
Einspeisevergütung in €/a: 256.23
Eigenverbrauch in kWh: 2510
kg CO₂/a: 220.4

Str

In [11]:
temp_2024 = temperatur_aussen.get_hourly_temperature(45257, 2024)

## Bochum

In [7]:
csv_path = "./2_Bochum/1_Jahr 1.11.23-31.11.24/Historical_c554ad32-286c-4c0a-aa07-0fc6db11a63f_01-11-2023_-_01-12-2024_1M_gridX.csv"
df_bochum = pd.read_csv(csv_path, skiprows=1, delimiter=",", decimal=",")

df_bochum['measuredAt'] = pd.to_datetime(df_bochum['measuredAt'])
df_bochum.set_index('measuredAt', inplace=True)


In [8]:
ergebnisse_bochum ={
    'pv_ertrag ': round(df_bochum['PV (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'einspeisung' : round(df_bochum['Gesamte Netzeinspeisung (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'netzbezug' : round(df_bochum['Gesamter Netzbezug (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'strombedarf': round(df_bochum['Haushalt (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'bs_strom' : round(df_bochum['Batterieladung (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'ev_strom' : round(df_bochum['EV (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'verbrauch' : round(df_bochum['Verbrauch (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
}

In [9]:
# Haus ähnlich zu Bochum simulieren
plz = 44787
anlage_groesse = 9.52
battery_capacity = 10.2
homeoffice = False
anzahl_personen = 4
strompreis = 0.358

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', flaeche, anzahl_personen, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(plz)
df = lastprofile_VDI4655.get_lastprofile(w, s, twe, flaeche, TRY_region, 2)
pv = pv_profil.get_pv_profil(plz, 2014, anlage_groesse)

import berechnen_ev
df_hems_bs, df_bs_ohne = berechnen_ev.mit_hems_bs(df.copy(), pv, battery_capacity, homeoffice)
ergebnisse_hems_bs = berechnen_ev.ersparnis_hems_bs(df_hems_bs, df_bs_ohne, anlage_groesse, strompreis)

KeyboardInterrupt: 

## Leverkusen

In [ ]:
csv_path = "./3_Leverkusen/1_Jahr 1.11.23-31.11.24/Historical_91d3ccb5-879d-4861-901e-820adb87f57c_01-11-2023_-_01-12-2024_1M_gridX.csv"
df_leverkusen = pd.read_csv(csv_path, skiprows=1, delimiter=",", decimal=",")

df_leverkusen['measuredAt'] = pd.to_datetime(df_leverkusen['measuredAt'])
df_leverkusen.set_index('measuredAt', inplace=True)

In [ ]:
ergebnisse_leverkusen ={
    'pv_ertrag ': round(df_leverkusen['PV (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'einspeisung' : round(df_leverkusen['Gesamte Netzeinspeisung (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'netzbezug' : round(df_leverkusen['Gesamter Netzbezug (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'strombedarf': round(df_leverkusen['Haushalt (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'bs_strom' : round(df_leverkusen['Batterieladung (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'wp_strom' : round(df_leverkusen['Wärmepumpe (Wh)'].fillna(0).astype(float).sum() / 1000, 2),
    'verbrauch' : round(df_leverkusen['Verbrauch (Wh)'].fillna(0).astype(float).sum() / 1000, 2)
}

In [ ]:
import importlib
importlib.reload(berechnen_wp)
# Haus ähnlich zu Leverkusen simulieren
plz = 51371
anlage_groesse = 10.16
battery_capacity = 7.7
anzahl_personen = 4
flaeche = 200
strompreis = 0.358

h, w, twe, s = lastprofile_VDI4655.get_jahresenergiebedarf('1984 - 1994', flaeche, anzahl_personen, 0)
TRY_region, T_n_aussen = try_region.get_try_t_n_aussen(plz)
df3 = lastprofile_VDI4655.get_lastprofile(w, s, twe, flaeche, TRY_region, 2)
df3['T_aussen'] = temperatur_aussen.get_hourly_temperature(plz, 2014)
hz, T_soll, T_n_vor, T_n_rueck = heizkurve.get_heizkurve('Heizkörper', df3['T_aussen'], T_n_aussen)
df3['T_vor'] = hz['T_vor']
wp_groesse, wp_modell = berechnen_wp.get_waermepumpe(h)
heizleistung_auslegung = heizkurve.get_heizleistung(T_n_aussen, h, T_soll)
df3 = heizkurve.get_cop(wp_groesse, df3)
V_ps, PS_verlust, Q_ps, Q_ps_max, Q_ps_ueber = berechnen_wp.get_pufferspeicher(h, T_n_vor, T_n_rueck)
pv = pv_profil.get_pv_profil(plz, 2014, anlage_groesse)

df_wp = berechnen_wp.get_max_heizleistung(wp_groesse, df3.copy())
df_1, df_ohne = berechnen_wp.mit_hems(df_wp.copy(), pv, Q_ps, Q_ps_max, Q_ps_ueber, PS_verlust)
df_2, df_ohne_2 = berechnen_wp.mit_hems_bsev(df_1, df_ohne, pv, battery_capacity, anlage_groesse, homeoffice)

ergebnisse_hems_lev = berechnen_wp.ersparnis_hems_bsev(df_2, df_ohne_2, anlage_groesse, strompreis)

## Wärmste Monat (August)